import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Văn hoá - Xã hội

find elements

In [2]:
# Function to crawl data from a page
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Get author and time information
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Get question title
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Get situation/summary
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Get detail link
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Send HTTP request to get detail page content
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Get answer from detail page
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Add article information to the list
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data

execute function

In [3]:
articles_data = []
for page_num in range(1, 300):
    page_url = f"https://chinhsachonline.chinhphu.vn/van-hoa-xa-hoi/t1/15/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

export to CSV

In [4]:
# Create DataFrame from the data list
df_bh = pd.DataFrame(articles_data)
df_bh.head(5)

,Name,Time,Question,Situation,Answer
0,Long Quy,09:05 30/07/2025,"Hồ sơ, thủ tục đăng ký sang tên xe",Tôi mua lại một chiếc xe máy SH đăng ký năm 20...,Bộ Công an trả lời vấn đề này như sau:Tại Điều...
1,Phạm Ngọc Dung,15:05 29/07/2025,Điều kiện về thực hành để cấp phép hành nghề d...,"Tôi là cử nhân dinh dưỡng, làm việc tại Trung ...",Bộ Y tế trả lời vấn đề này như sau:Tại khoản 4...
2,Phạm Khánh Trung,07:05 29/07/2025,Đơn vị nào cung ứng dịch vụ sự nghiệp thông ti...,Theo Công văn số 68/CV-BCĐ ngày 28/5/2025 của ...,"Cục Thông tin cơ sở và Thông tin đối ngoại, Bộ..."
3,Nguyễn Thanh Thúy,08:05 26/07/2025,Phụ cấp thâm niên nhà giáo được tính từ khi nào?,Tôi trúng tuyển viên chức ngành giáo dục tháng...,Bộ Giáo dục và Đào tạo trả lời vấn đề này như ...
4,Lê Thắm,08:26 25/07/2025,Văn bằng nào phù hợp để giảng dạy môn Giáo dục...,Tôi có bằng thạc sĩ quản lý công của Học viện ...,Bộ Giáo dục và Đào tạo trả lời vấn đề này như ...


In [5]:

 # Save DataFrame to CSV file
csv_file = '../source_fine_tuning/data/vh_xh_data.csv'
df_bh.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../source_fine_tuning/data/vh_xh_data.csv


In [6]:
df_bh=pd.read_csv("../source_fine_tuning/data/vh_xh_data.csv",encoding='utf-8')

In [7]:
df_bh.shape

(2002, 6)